# San Francisco Crime prediction 
# Based on 2 layer neural net and count featurizer

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from copy import deepcopy
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin

%matplotlib inline  

Import data

In [ ]:
trainDF=pd.read_csv("data/train.csv")
#trainDF=trainDF.sample(15000)

Clean up wrong X and Y values (very few of them)

In [ ]:
labels = trainDF['Category']
sss = StratifiedShuffleSplit(labels, train_size=0.5,random_state=0)
for train_index, dev_index in sss:
    train_data,dev_data=trainDF.iloc[train_index],trainDF.iloc[dev_index]
    train_labels,dev_labels=labels[train_index],labels[dev_index]
dev_data.index=range(len(dev_data))
train_data.index=range(len(train_data))
train_labels.index=range(len(train_labels))
dev_labels.index=range(len(dev_labels))
labels.index=range(len(labels))
trainDF.index=range(len(trainDF))
        


In [ ]:
def parse_time(x):
    DD=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
    time=DD.hour#*60+DD.minute
    day=DD.day
    month=DD.month
    year=DD.year
    return time,day,month,year

def get_season(x):
    summer=0
    fall=0
    winter=0
    spring=0
    if (x in [5, 6, 7]):
        summer=1
    if (x in [8, 9, 10]):
        fall=1
    if (x in [11, 0, 1]):
        winter=1
    if (x in [2, 3, 4]):
        spring=1
    return summer, fall, winter, spring

In [ ]:
class RemoveColumnsTransformer(TransformerMixin):
    
    def __init__(self,cols=[],include=True):
        self.cols = cols
        self.include = include

    def transform(self, X,y=None, **transform_params):
        print("Dropping " , self.cols)
        if self.include:
            x_num_train = X.loc[:,self.cols]
            
        else:
            x_num_train = X.drop( self.cols, axis = 1,errors='ignore') 
        return X
      
    def fit(self, X, y=None, **fit_params):
        return self

class ScalerTransform(TransformerMixin):
    
    def __init__(self,cols=[],include=True):
        self.cols = cols
        self.include = include

    def transform(self, X,y=None, **transform_params):
        print("Scaler")
        if self.include:
            x_num_train = X.loc[:,self.cols]
        else:
            x_num_train = X.drop( self.cols, axis = 1 )
        return self.scaler.transform(x_num_train)
      
    def fit(self, X, y=None, **fit_params):
        if self.include:
            x_num_train = X[self.cols]
        else:
            x_num_train = X.drop( self.cols, axis = 1 )      
        self.scaler = preprocessing.StandardScaler().fit(x_num_train)
        return self


class DatesTransformer(TransformerMixin):

    def __init__(self,datecol="Dates"):
        self.datecol = datecol

    
    def transform(self, X, **transform_params):
        if(self.datecol in X):
            print("Dates")
            DD =X[self.datecol].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
            time,day,month,year  =  DD.dt.hour, DD.dt.day,DD.dt.month,DD.dt.year  
            return np.array([time,day,month,year]).T
        return 
        

    def fit(self, X, y=None, **fit_params):
        return self

class SeasonsTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        print("Seasons")

        def get_season(x):
            summer=0
            fall=0
            winter=0
            spring=0
            if (x in [5, 6, 7]):
                summer=1
            if (x in [8, 9, 10]):
                fall=1
            if (x in [11, 0, 1]):
                winter=1
            if (x in [2, 3, 4]):
                spring=1
            return summer, fall, winter, spring
        
        if('Dates' in X):
            DD =X.loc[:,'Dates'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
            awake=DD.dt.hour.apply(lambda x: 1 if (x==0 or (x>=8 and x<=23)) else 0)
            summer, fall, winter, spring=zip(*DD.dt.month.apply(get_season))
            return  np.array([awake,summer,fall,winter,spring]).T     

    def fit(self, X, y=None, **fit_params):
        return self
    
class AddIsIntersectionTransformer(TransformerMixin):

    def transform(self, X, **transform_params):
        print("Intersection")

        if('Address' in X):
            return (X.loc[:,"Address"].apply(lambda x: 1 if "/" in x else 0))[:,None]

    def fit(self, X, y=None, **fit_params):
        return self    
    
class StreetNamesTransformer(TransformerMixin):

    def transform(self, X, **transform_params):
        def getstreet(x):
            if  "/" in x: 
                return pd.Series(list(re.findall(r'([A-Z0-9 ]+)\s/\s([A-Z0-9 ]+)', x)[0]))
            else: 
                street = re.findall(r'\d+.*?\s+Block of ([A-Z0-9 ]+)', x)[0]
                return pd.Series([street, street])
        
        print("StreetNames")
        streets = X["Address"].apply(getstreet)
        streets.columns = ["street1","street2"]
        return pd.concat([X2, streets], axis=1)

    def fit(self, X, y=None, **fit_params):
        return self    
    
class LogOddsTransformer(TransformerMixin):
    def __init__(self,togroup):
        self.togroup = togroup
        

        
    def transform(self, X, **transform_params):
        print "LogOdds transform"
        col = X[self.togroup] 
        address_features=col.apply(lambda x: self.logodds.setdefault(tuple(x) if len(x) > 1 else x[0],self.default_logodds),axis=1)
        PA = col.apply(lambda x: self.logoddsPA.setdefault(tuple(x) if len(x) > 1 else "",0),axis=1)
        return np.hstack((address_features,PA[:,np.newaxis]))


    def fit(self, X, y=None, **fit_params):
        print("LogOdds")
        categories=sorted(y.unique())
        X2 = X.assign(Category=y.astype('object'))
        C_counts=X2.groupby(["Category"]).size()
        A_C_counts=X2.groupby(self.togroup +["Category"]).size()
        A_counts=X2.groupby(self.togroup).size()
        addresses=A_counts.keys()
        logodds={}
        logoddsPA={}
        MIN_CAT_COUNTS=2
        default_logodds=np.log(C_counts/len(X2))-np.log(1.0-C_counts/float(len(X2)))
        for addr in addresses:
            PA=A_counts[addr]/float(len(X2))
            logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
            logodds[addr]=deepcopy(default_logodds)
            for cat in A_C_counts[addr].keys():
                if (A_C_counts[addr][cat]>MIN_CAT_COUNTS) and A_C_counts[addr][cat]<A_counts[addr]:
                    PA=A_C_counts[addr][cat]/float(A_counts[addr])
                    logodds[addr][cat]=np.log(PA)-np.log(1.0-PA)
            #logodds[addr]=pd.Series(logodds[addr])
            #logodds[addr].index=range(len(categories))
        self.logodds=logodds
        self.default_logodds = default_logodds
        self.logoddsPA=logoddsPA
        return self       
    
class MarkDuplicatesTransformer(TransformerMixin):
    def __init__(self,cols,include=True):
        self.cols = cols
        self.include = include

    def transform(self, X, **transform_params):  
        print("Duplicates")
        if self.include:
            X2 = X[self.cols]
        else:
            X2 = X.drop( self.cols, axis = 1 ,errors='ignore')
        return (pd.Series(X2.duplicated()|X2.duplicated(keep="last")).apply(int))[:,None]

    def fit(self, X, y=None, **fit_params):
        return self        



    
class FeatureVecotrizer(TransformerMixin):
    def __init__(self,cols,include=True):
        self.cols = cols
        self.include = include

    def transform(self, X,y=None, **transform_params):
        print("Vectorizer")
        if self.include:
            x_cat_train = X[self.cols]
            x_num_train = X.drop( self.cols, axis = 1 )
        else:
            x_num_train = X[self.cols]
            x_cat_train = X.drop( self.cols, axis = 1 )
        
        x_cat_train.fillna( 'NA', inplace = True )
        vec_x_cat_train  =pd.get_dummies(x_cat_train)
        return vec_x_cat_train

    def fit(self, X,y=None, **fit_params):
        x_cat_train = X[self.cols]
        x_cat_train.fillna( 'NA', inplace = True )
        x_cat_train = x_cat_train.T.to_dict().values()
        return  self
    


In [ ]:
class PapaDocNeuralNetworkModel(Sequential):
    
    def __init__(self,hn=32,dp=0.5,layers=1,epochs=1,batches=64,verbose=0,*args,**kwargs):
        self.hn = hn
        self.dp = dp
        self.Nlayers = layers
        self.epochs=epochs
        self.batches = batches
        self.verbose = verbose
        super(PapaDocNeuralNetworkModel,self).__init__(*args,**kwargs)
    
    def fit(self,X_train,y_train,X_test=None,y_test=None,*args,**kwargs):
        y_train=y_train.astype('category')
        input_dim=X_train.shape[1]
        output_dim=len(y_train.unique())
        Y_train=y_train.cat.rename_categories(range(len(y_train.unique())))
        self.add(Dense(input_dim=input_dim, output_dim=self.hn, init='glorot_uniform'))
        self.add(PReLU(input_shape=(self.hn,)))
        self.add(Dropout(self.dp))
        for i in range(self.Nlayers):
            self.add(Dense(input_dim=self.hn, output_dim=self.hn, init='glorot_uniform'))
            self.add(PReLU(input_shape=(self.hn,)))
            self.add(BatchNormalization())
            self.add(Dropout(self.dp))

        self.add(Dense(input_dim=self.hn, output_dim=output_dim, init='glorot_uniform'))
        self.add(Activation('softmax'))
        self.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
        if X_test is not None:
            X_test=X_test.as_matrix()
            y_test=y_test.astype('category')
            Y_test=np_utils.to_categorical(y_test.cat.rename_categories(range(len(y_test.unique()))))
            super(PapaDocNeuralNetworkModel,self).fit(X_train, Y_train, nb_epoch=self.epochs, \
                            batch_size=self.batches,verbose=self.verbose,\
                              validation_data=(X_test,Y_test),*args,**kwargs)
        else:
            super(PapaDocNeuralNetworkModel,self).fit(X_train, Y_train, nb_epoch=self.epochs, \
                                                      batch_size=self.batches,verbose=self.verbose,*args,**kwargs)
        return self


In [ ]:


class EnsembleClassifier(BaseEstimator, ClassifierMixin):
    """
    Ensemble classifier for scikit-learn estimators.

    Parameters
    ----------

    clf : `iterable`
      A list of scikit-learn classifier objects.
    weights : `list` (default: `None`)
      If `None`, the majority rule voting will be applied to the predicted class labels.
        If a list of weights (`float` or `int`) is provided, the averaged raw probabilities (via `predict_proba`)
        will be used to determine the most confident class label.

    """
    def __init__(self, clfs, weights=None):
        self.clfs = clfs
        self.weights = weights

    def fit(self, X, y):
        print("Ensemble")
        """
        Fit the scikit-learn estimators.

        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]
            Training data
        y : list or numpy array, shape = [n_samples]
            Class labels

        """
        for clf in self.clfs:
            clf.fit(X, y)

    def predict(self, X):
        """
        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]

        Returns
        ----------

        maj : list or numpy array, shape = [n_samples]
            Predicted class labels by majority rule

        """

        self.classes_ = np.asarray([clf.predict(X) for clf in self.clfs])
        if self.weights:
            avg = self.predict_proba(X)

            maj = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=avg)

        else:
            maj = np.asarray([np.argmax(np.bincount(self.classes_[:,c])) for c in range(self.classes_.shape[1])])

        return maj

    def predict_proba(self, X):

        """
        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]

        Returns
        ----------

        avg : list or numpy array, shape = [n_samples, n_probabilities]
            Weighted average probability for each class per sample.

        """
        self.probas_ = [clf.predict_proba(X) for clf in self.clfs]
        avg = np.average(self.probas_, axis=0, weights=self.weights)

        return avg

In [ ]:
N_EPOCHS=20
N_HN=128
N_LAYERS=1
DP=0.5

In [ ]:
papadocFeatures =  Pipeline([("Features",FeatureUnion([
                    ("XY",ScalerTransform(["X","Y"])), \
                    ("DateToTime",DatesTransformer()), \
                    ("Intersection",AddIsIntersectionTransformer()),\
                    ("Vectorizer",FeatureVecotrizer(['PdDistrict','DayOfWeek'])),\
                    #("LogsOdds",Pipeline([\
                                #("Street",StreetNamesTransformer()),\
                                #("LogOdds",LogOddsTransformer(["PdDistrict","street1","street2"]))\
                    #])),\
                    ("LogOdds",LogOddsTransformer(["Address"])),\

                    ("MarkDuplicates",MarkDuplicatesTransformer(["PdDistrict","DayOfWeek","Dates","Address"])),\
                    ("Seasons",SeasonsTransformer())
                    ],n_jobs=3)),                   
                    ("ScaleEverything",preprocessing.StandardScaler())])



ensemblepipe = Pipeline([("Features",papadocFeatures),\
               ('eclf', EnsembleClassifier([LogisticRegression(C=.01),
                    PapaDocNeuralNetworkModel(hn=N_HN,layers=N_LAYERS,epochs=N_EPOCHS,verbose=2,dp=DP)
]))])

model = ensemblepipe.fit(train_data,train_labels.astype('category'))



In [ ]:
print "train", log_loss(train_labels.astype('category'), model.predict_proba(train_data))
print "test", log_loss(dev_labels.astype('category'), model.predict_proba(dev_data))

In [ ]:
testDF=pd.read_csv("data/test.csv")
testDF.index=range(len(testDF))

In [ ]:
model = ensemblepipe.fit(trainDF,labels.astype('category'))


In [ ]:
predDF=pd.DataFrame(model.predict_proba(testDF),columns=sorted(labels.unique()))

In [ ]:
predDF.head()

In [ ]:
predDF.to_csv("crimeSF_NN_logodds.csv",index_label="Id",na_rep="0")